In [1]:
### Test Database Connection, Create, Insert
import pandas as pd
import geopandas as gpd
import supabase_py
import psycopg2
import sqlalchemy
import os
from dotenv import load_dotenv

In [2]:
### Establish Connection to Supabase
### Connect to the database
# downloaded certiticate
# Set connection details in .env

# Load environment variables from .env file
load_dotenv()

# Get connection parameters from environment variables
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
user_read_only = os.getenv("DB_USER_READONLY")
password = os.getenv("DB_PASSWORD")
password_read_only = os.getenv("DB_PASSWORD_READONLY")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
ssl_cert_path = os.getenv("SSL_CERT_PATH")

# Construct the connection string
conn_str = f"dbname={dbname} user={user} password={password} host={host} port={port} sslmode=require sslrootcert={ssl_cert_path}"

# Etablish connection object

#conn.close()
conn = psycopg2.connect(conn_str)
print(conn)

<connection object at 0x7fa71395e340; dsn: 'user=postgres password=xxx dbname=postgres host=db.cnpmlxzpwbfkunigsvsg.supabase.co port=5432 sslmode=require sslrootcert=./supabase_goal100_certificate.crt', closed: 0>


In [ ]:
sql_create = """CREATE TABLE public.test_create_2 (
    id bigint generated by default as identity primary key,
    test_num float8)
    ;
    """
sql_insert = "INSERT INTO public.test_create (test_num) VALUES (1.0);"

In [ ]:
### New role goal100_internal_analyst created with UI (only login enabled)
sql_role_privileges = """-- Internal analyst can login
GRANT CONNECT ON DATABASE postgres TO goal100_internal_analyst;
-- Internal analyst can work within schema public
GRANT USAGE ON SCHEMA public TO goal100_internal_analyst;
-- Internal analyst can perform select queries (read data) in schema public
GRANT SELECT ON ALL TABLES IN SCHEMA public TO goal100_internal_analyst;
-- Internal analyst can perform select on all new tables in schema public
ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO goal100_internal_analyst;"""

In [ ]:
### Create User under Role goal100_internal_analyst
sql_user = f"""CREATE USER {user_read_only} WITH PASSWORD '{password_read_only}';
GRANT goal100_internal_analyst TO {user_read_only};
"""

In [ ]:

conn_cursor = conn.cursor()
### Create simple test table
### Table created 
### Insert Data - via psycopg2
conn_cursor.execute(sql_user)
### Look up in supabase - data inserted
conn.commit()
conn_cursor.close()
conn.close()


In [4]:
### Try connecting with read-only user
conn_str = f"dbname={dbname} user={user_read_only} password={password_read_only} host={host} port={port} sslmode=require sslrootcert={ssl_cert_path}"
conn = psycopg2.connect(conn_str)
### Connection worked

In [ ]:
### Define SQL Execute as a function for selects and changes
def sql_execute(sql_query, conn_db):
    try:
        # Create a cursor
        cur = conn_db.cursor()
    
        # Execute your SQL statement
        cur.execute(sql_query)
    
        # Commit the transaction
        if sql_query.startswith("SELECT"): 
            df_return = pd.DataFrame(cur.fetchall())
            
        else:
            conn.commit()
            
    except Exception as e:
        # Handle the exception
        print(f"Error: {e}")
        conn.rollback()
    
    finally:
        # Close the cursor and connection
        cur.close()
        if 'df_return' in locals():
            return df_return
        return None
        #conn.close()

In [ ]:
print(conn)
### Test privilges: Select on windpower and create new table and insert into existing table
sql_create = """CREATE TABLE public.test_create_3 (
    id bigint generated by default as identity primary key,
    test_num float8)
    ;
    """
sql_insert = "INSERT INTO public.test_create (test_num) VALUES (1.0);"
sql_select = "SELECT * FROM public.wind_extended LIMIT 10;"

In [ ]:
sql_execute(sql_create,conn)
# Error: permission denied for schema public
# LINE 1: CREATE TABLE public.test_create_3 (

In [ ]:
sql_execute(sql_insert,conn)
# Error: permission denied for table test_create


In [5]:
df_test = sql_execute(sql_select)
# empty list returned for 

[]